In [20]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [21]:
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

from spooky import *

In [22]:
PATH = 'data/spooky'

os.makedirs(f'{PATH}/models', exist_ok=True)
os.makedirs(f'{PATH}/tmp', exist_ok=True)

bs = 64
bptt = 70

emb_sz = 400       # size of each embedding vector
nh = 1024           # of hidden activations per layer
nl = 3             # of layers

# for NLP, configure Adam to use less momentum than the defaul of 0.9
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [23]:
# get raw training and test datasets
train_raw_df = pd.read_csv(f'{PATH}/train.csv')
test_df = pd.read_csv(f'{PATH}/test.csv')

len(train_raw_df), len(test_df)

(19579, 8392)

In [24]:
# build train and validation datasets
val_idxs = get_cv_idxs(len(train_raw_df), val_pct=0.05)

train_df =  train_raw_df.drop(val_idxs)
val_df = train_raw_df.iloc[val_idxs]

len(train_df), len(val_df), len(test_df)

(18601, 978, 8392)

In [25]:
# tokenize = split each sentence into a list of words
' '.join(spacy_tok(train_df.text.iloc[0]))

'This process , however , afforded me no means of ascertaining the dimensions of my dungeon ; as I might make its circuit , and return to the point whence I set out , without being aware of the fact ; so perfectly uniform seemed the wall .'

In [26]:
#createa torchtext field = describes how to preprocess a piece of text
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [27]:
FILES = dict(train_df=train_df, val_df=val_df, test_df=test_df)

# min_freq = 10 says, "treat any word that appears less than 10 times as the word <unk>"
md = LanguageModelData.from_dataframes(PATH, TEXT, 'text', **FILES, bs=bs, bptt=bptt, min_freq=10)

In [28]:
# after building the ModelData object, TEXT.vocab is set.  because this will be needed again, save it
pickle.dump(TEXT, open(f'{PATH}/models/TEXT.pkl', 'wb'))

In [29]:
# batches, # of unique tokens in vocab, # of items in ds, # of words in ds
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(125, 4755, 1, 565792)

In [30]:
# int to string mapping
TEXT.vocab.itos[:12]

['<unk>', '<pad>', ',', 'the', 'of', '.', 'and', 'to', 'i', 'a', 'in', 'was']

In [31]:
# string to int mapping
TEXT.vocab.stoi['the']

3

In [32]:
# in a LanguageModelData object there is only one item in each dataset: all the words joined together
md.trn_ds[0].text[:12]

['this',
 'process',
 ',',
 'however',
 ',',
 'afforded',
 'me',
 'no',
 'means',
 'of',
 'ascertaining',
 'the']

In [33]:
# torchtext will handle turning this words into integer Ids
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
   31
 2949
    2
  151
    2
 1431
   27
   42
  301
    4
    0
    3
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

In [34]:
batch = next(iter(md.trn_dl))
print(batch[0].size()), print(batch[1].size())

batch

torch.Size([76, 64])
torch.Size([4864])


(Variable containing:
    31     6    52  ...      2     3     2
  2949     0     0  ...      3     0     0
     2     7     2  ...      0     7     2
        ...          ⋱          ...       
    28     2     4  ...      7    20   320
    24  3289     3  ...   1074     0     2
     2   216     0  ...     26     5    10
 [torch.cuda.LongTensor of size 76x64 (GPU 0)], Variable containing:
  2949
     0
     0
   ⋮  
   338
   325
     3
 [torch.cuda.LongTensor of size 4864 (GPU 0)])

In [35]:
learner = md.get_model(opt_fn, emb_sz, nh, nl,
                      dropouti=0.1, dropout=0.1, wdrop=0.2, dropoute=0.04, dropouth=0.1)

learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip = 0.3

In [ ]:
lrf = learner.lr_find() 

In [ ]:
learner.sched.plot()

In [36]:
lr = 1e-3

In [37]:
learner.fit(lr, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

  6%|▋         | 8/125 [00:02<00:39,  2.97it/s, loss=8.46]

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



[ 0.       5.76916  5.64622]                                
[ 1.       5.41764  5.26515]                                
[ 2.       5.23646  5.1827 ]                                
[ 3.       5.11374  4.99851]                                
[ 4.       4.98249  4.88693]                                
[ 5.       4.89676  4.84356]                                
[ 6.       4.85656  4.83048]                                
[ 7.       4.84061  4.74867]                                
[ 8.       4.74853  4.70073]                                
[ 9.       4.66462  4.63213]                                
[ 10.        4.58969   4.57953]                             
[ 11.        4.52939   4.55109]                             
[ 12.        4.49072   4.5317 ]                             
[ 13.        4.45834   4.52138]                             
[ 14.        4.45779   4.51699]                             



In [38]:
learner.save_encoder('spooky_adam_enc1')
# learner.load_encoder('spooky_adam1_enc')

In [ ]:
learner.fit(lr, 1, wds=1e-6, cycle_len=10, cycle_save_name='spooky_adam_enc2_c1_cl10')

In [41]:
# learner.load_cycle('spooky_adam_enc2_c2_cl10',0)

In [42]:
learner.save_encoder('spooky_adam_enc2')
# learner.load_encoder('spooky_adam2_enc')

In [43]:
learner.fit(lr/2, 1, wds=1e-6, cycle_len=10, cycle_save_name='spooky_adam_enc3_c1_cl10')

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))


  0%|          | 0/125 [00:00<?, ?it/s]

  1%|          | 1/125 [00:00<00:53,  2.33it/s, loss=3.9]

Exception in thread Thread-21:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




 53%|█████▎    | 66/125 [00:23<00:21,  2.76it/s, loss=4.05]


100%|██████████| 125/125 [00:45<00:00,  2.74it/s, loss=4.07]
                                                            [ 0.       4.06726  4.43997]

  3%|▎         | 4/125 [00:01<00:52,  2.31it/s, loss=4.07]


...

100%|██████████| 125/125 [00:45<00:00,  2.73it/s, loss=3.81]
                                                            [ 7.       3.81475  4.47228]

 25%|██▍       | 31/125 [00:11<00:33,  2.80it/s, loss=3.81]


 77%|███████▋  | 96/125 [00:34<00:10,  2.76it/s, loss=3.8]


100%|██████████| 125/125 [00:45<00:00,  2.75it/s, loss=3.79]
                                                            [ 8.       3.78935  4.48984]

 27%|██▋       | 34/125 [00:12<00:33,  2.75it/s, loss=3.8] 


 79%|███████▉  | 99/125 [00:36<00:09,  2.73it/s, loss=3.79]


100%|██████████| 125/125 [00:45<00:00,  2.74it/s, loss=3.78]
                                                            [ 9.       3.77848  4.47463]



In [44]:
learner.save_encoder('spooky_adam_enc3')

In [46]:
learner.load_encoder('spooky_adam_enc2')

In [47]:
# metric perplexity (how language model accuracy generally measured) = exp() of loss functino
np.exp(4.33935)

76.657695638682966

In [48]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True)

# get folds
kfolds = [ (train_idxs, val_idxs) for train_idxs, val_idxs in skf.split(train_df.id, train_df.author) ]
print(len(kfolds))

5


## Test

In [49]:
# create a short bit of text to "prime" the precitions, then use torchtext to numericalize it
# so we can feed it into our language model
m = learner.model
ss = """. It was a dark and scary night. The old"""
s = [spacy_tok(ss)]
t = TEXT.numericalize(s)
' '.join(s[0])

'. It was a dark and scary night . The old'

In [50]:
m[0].bs = 1      # set batch size = 1
m.eval()         # turn-off dropout
m.reset()        # reset hidden state
res, *_ = m(t)   # get predictions from model
m[0].bs = bs     # put batch size back to what it was

In [51]:
# top 10 predictions for next word
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['<unk>',
 'man',
 'things',
 'men',
 'old',
 'and',
 'whateley',
 ',',
 'people',
 'houses']

In [52]:
# try to generate more text
print(ss, "\n")

for i in range(50):
    n = res[-1].topk(2)[1]
    n = n[1] if n.data[0] == 0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res, *_ = m(n[0].unsqueeze(0))
    
print('...')

. It was a dark and scary night. The old 

man , who had been a little man , had been the most remarkable and most remarkable of the world . i had been a little , and had not been able to get out of the world . i was not , however , to be sure , but ...


## Predict the author

In [53]:
bs = 64
bptt = 70

emb_sz = 400       # size of each embedding vector
nh = 1024           # of hidden activations per layer
nl = 3             # of layers

# for NLP, configure Adam to use less momentum than the defaul of 0.9
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [54]:
# use the same vocab built from the language model so as to ensure words map to same Ids
TEXT = pickle.load(open(f'{PATH}/models/TEXT.pkl', 'rb'))

In [55]:
AUTHOR_LABEL = data.Field(sequential=False)
splits = SpookyDataset.splits(TEXT, AUTHOR_LABEL, train_df, val_df, test_df)

In [56]:
t = splits[0].examples[0]

In [57]:
t.label, ' '.join(t.text[:10])

('EAP', 'this process , however , afforded me no means of')

In [58]:
# fastai can create a ModelData object directly from torchtext splits
md2 = TextData.from_splits(PATH, splits, bs)

In [59]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=emb_sz, n_hid=nh, n_layers=nl,
                      dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)

m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'spooky_adam_enc2')

In [60]:
m3.clip = 25.
lrs = np.array([1e-4, 1e-3, 1e-2])

In [61]:
m3.freeze_to(-1) # freeze everything except last layer
m3.fit(lrs/2, 2, metrics=[accuracy])

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))


 31%|███       | 89/290 [00:06<00:14, 13.85it/s, loss=0.905]


 35%|███▍      | 101/290 [00:06<00:13, 14.45it/s, loss=0.894][A

Exception in thread Thread-38:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



[ 0.       0.87596  0.74771  0.67917]                        
[ 1.       0.80553  0.74185  0.68854]                        



In [62]:
m3.unfreeze()
m3.fit(lrs, 2, metrics=[accuracy], cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       0.64033  0.57021  0.76771]                        
[ 1.       0.54842  0.51538  0.79167]                        



In [63]:
m3.fit(lrs/2, 4, metrics=[accuracy], cycle_len=1, cycle_mult=2, cycle_save_name='spooky_sent1_c4_cl1x2')

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

[ 0.       0.48122  0.47413  0.80833]                        
[ 1.       0.46605  0.46353  0.81771]                        
[ 2.       0.38426  0.45132  0.82083]                        
[ 3.       0.42502  0.46889  0.80729]                        
[ 4.       0.35398  0.46011  0.81667]                        
[ 5.       0.28249  0.45191  0.82292]                        
[ 6.       0.26402  0.46145  0.82188]                        
[ 7.       0.31668  0.4834   0.82292]                        
[ 8.       0.29387  0.4616   0.82396]                        
[ 9.       0.25343  0.48163  0.82604]                        
[ 10.        0.20527   0.51006   0.82396]                    
[ 11.        0.17881   0.49798   0.83958]                    
[ 12.        0.15088   0.53388   0.83125]                    
[ 13.        0.12904   0.53637   0.83229]                    
[ 14.        0.13446   0.53578   0.83542]                    



In [64]:
m3.fit(lrs/4, 3, metrics=[accuracy], cycle_len=3, cycle_save_name='spooky_sent2_c3_cl3')

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

[ 0.       0.16886  0.51518  0.83229]                        
[ 1.       0.11597  0.54311  0.83542]                        
[ 2.       0.10753  0.54276  0.84062]                        
[ 3.       0.14199  0.56554  0.83229]                        
[ 4.       0.11287  0.57523  0.83646]                        
[ 5.       0.10518  0.58223  0.8375 ]                         
[ 6.       0.13167  0.60857  0.83958]                        
[ 7.       0.10147  0.61336  0.83958]                         
[ 8.       0.08056  0.62719  0.84062]                         



In [83]:
m3.load_cycle('spooky_sent1_c4_cl1x2', 1) # NOTE: using model with lower val loss is better
# m3

In [84]:
classes = AUTHOR_LABEL.vocab.itos
classes

['<unk>', 'EAP', 'MWS', 'HPL']

In [85]:
preds = []

m = m3.model 
m[0].bs = 1
for index, row in test_df.iterrows():
    ss = row['text']
    s = [spacy_tok(ss)]
    t = TEXT.numericalize(s)
   
    m.eval()
    m.reset()
    res,*_ = m(t)
    preds.append(to_np(res).squeeze()[1:])
#     preds.append(to_np(res).squeeze())
    
preds = np.array(preds)
preds.shape

(8392, 3)

In [86]:
probs = to_np(F.softmax(torch.from_numpy(preds)))

In [87]:
probs.shape

(8392, 3)

In [88]:
def do_clip(arr, mx):
    clipped = np.clip(arr, (1-mx)/1, mx)
    return clipped/clipped.sum(axis=1)[:, np.newaxis]

In [89]:
# probs = do_clip(probs, 0.98)

In [90]:
preds_test_df = test_df.copy()
preds_test_df['EAP'] = probs[:,0]
preds_test_df['MWS'] = probs[:,1]
preds_test_df['HPL'] = probs[:,2]

preds_test_df.drop('text', axis=1, inplace=True)
preds_test_df.head()

,id,EAP,MWS,HPL
0,id02310,0.027351,2.754847e-01,6.971645e-01
1,id24541,1.000000,1.474728e-07,4.589602e-09
2,id00134,0.000164,1.170791e-03,9.986647e-01
3,id27757,0.698621,7.681423e-04,3.006105e-01
4,id04081,0.695955,2.278781e-01,7.616729e-02


In [91]:
preds_test_df.to_csv(f'{PATH}/subm_wg_20171127_4.csv', index=None)

In [ ]:
preds_test_df = pd.read_csv(f'{PATH}/subm_wg_20171126_3.csv', index_col=None)

In [ ]:
preds_test_df.head()

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(f'{PATH}/subm_wg_20171126_3.csv')